In [257]:
%run "src/nn/factorVAE.py"

Hidden Features shape : torch.Size([40, 32])
Portoflio shape : torch.Size([40, 100])
mu_post shape : torch.Size([1, 64]) , sigma_post shape : torch.Size([1, 64])
A post factor sample : tensor([[-0.0251,  1.0134,  0.9552, -0.7933, -0.3174, -0.3779, -0.0963, -0.0579,
         -0.2038,  0.4169,  0.8579, -0.0513,  0.7206,  0.1310,  0.2898,  0.3320,
          0.2841,  0.4943, -0.3019, -0.8400, -0.1451,  0.5816, -0.3454,  0.6914,
          1.0123,  0.2815,  0.5450,  0.6407,  0.3073, -0.4669,  0.0851,  0.5037,
          1.2655, -1.7337,  0.2668,  0.0826, -1.1659, -0.2792,  1.0265,  0.5371,
          0.4262,  0.3798, -1.1599, -0.1375, -0.7519,  0.1956, -0.1297,  0.0871,
          0.7342,  0.7079,  0.2127, -0.0476, -0.7535,  0.0696,  0.6441,  0.7256,
         -1.3062,  0.1730, -0.1454, -0.1280,  1.9228,  0.0137,  1.1164, -0.3767]],
       grad_fn=<AddBackward0>) 
Beta shape : torch.Size([40, 64])
mu_alpha shape : torch.Size([40, 1]) , sigma_alpha shape : torch.Size([40, 1])
stock_returns shape 